In [90]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [91]:
batch_size = 64
num_classes = 10
learnin_rate = 0.0001
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [92]:
print(torch.cuda.is_available())

print(torch.__version__)

True
2.1.2+cu121


In [93]:
dataset = torch.load('images.pth')
labels = torch.load('labels.pth')

In [94]:
class handDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.x = dataset
        self.y = labels
        self.n_samples = len(dataset)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [95]:
generator = torch.Generator().manual_seed(42)

train_size = int(len(dataset) * .7)
test_size = len(dataset) - train_size

hand_dataset = handDataset()
train_dataset, test_dataset = torch.utils.data.random_split(hand_dataset, [train_size, test_size], generator=generator)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [96]:
class FF_model(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3,500) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, num_classes)  
    
    def forward(self, x):
        out = self.flatten(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [97]:
class CNN_model(nn.Module):

    def __init__(self, num_classes):
        super(CNN_model, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(14*14*64,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
        
        out = self.conv_layer2(out)
        out = self.max_pool2(out)

        out = self.flatten(out)
        #896x14 and 12544x128)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)

        return out


In [98]:
cnn_model = CNN_model(num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(cnn_model.parameters(), lr=learnin_rate, weight_decay=0.005, momentum=0.9)

total_step = len(train_loader)

In [99]:
for epoch in range(num_epochs):

    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images=images.to(device)
        labels=labels.to(device)
        
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

438it [00:03, 131.20it/s]


Epoch [1/10], Loss: 2.2728


438it [00:02, 148.17it/s]


Epoch [2/10], Loss: 2.2436


438it [00:02, 148.11it/s]


...

Epoch [8/10], Loss: 0.5143


438it [00:02, 147.06it/s]


Epoch [9/10], Loss: 0.4545


438it [00:02, 147.38it/s]

Epoch [10/10], Loss: 0.2909


In [100]:
with torch.no_grad():
    correct = 0
    total = 0
    start = timer()
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        labels_class=torch.max(labels,1)
        print(labels_class)
        for i,sample in enumerate(labels_class[1]):
            print(sample)
            print(predicted[i])
            if sample==predicted[i]:
                correct += 1
    end=timer()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))
    print("TIme of inference =" + str((end-start)/total) +"s")

torch.return_types.max(
values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'),
indices=tensor([2, 4, 2, 9, 6, 6, 5, 1, 4, 5, 5, 2, 5, 1, 8, 3, 5, 8, 4, 2, 1, 8, 4, 6,
        4, 4, 7, 9, 1, 4, 6, 9, 2, 2, 3, 3, 4, 1, 5, 5, 4, 5, 5, 9, 3, 3, 6, 9,
        7, 3, 1, 1, 4, 8, 0, 1, 8, 1, 0, 2, 5, 3, 7, 5], device='cuda:0'))
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(4, device='cuda:0')
tensor(4, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(4, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, devi